# Exploratory Data Analysis and Preprocessing

Author: Marco Pellegrino<br>
Year: 2024

This overall project aims to build a simple model to predict the probability of loan default based on loan application data. This information helps assess business risk and improve loan approval decisions.

In this notebook, data is explored and formatted, ready for the processing and training pipeline.

In [ ]:
import requests
import pandas as pd
import datetime
import seaborn as sns
import matplotlib.pyplot as plt
import time
import numpy as np
import os

# Import paths
from config import *

In [ ]:
# Check if the directory for plots exists
if not os.path.exists(PATH_PLOTS_RAW):
    # If it doesn't exist, create the directory
    os.makedirs(PATH_PLOTS_RAW)

# Load Data

#### a) Import data from local CSV file

In [ ]:
# Import data
df = pd.read_csv(PATH_DATA_RAW+'loan_application_data-raw.csv', index_col=False)

#### b) Import data from AWS S3 bucket

In [ ]:
def read_csv_from_s3(bucket_name, region_name):
    """
    Read a CSV file from S3 and return a Pandas DataFrame.

    Args:
        bucket_name (str): The name of the S3 bucket.
        region_name (str): AWS region name.

    Returns:
        pd.DataFrame: The Pandas DataFrame containing the CSV data.
    """
    # Create a connection to S3
    s3 = boto3.client('s3',
                      aws_access_key_id=os.environ.get('AWS_ACCESS_KEY_ID'),
                      aws_secret_access_key=os.environ.get('AWS_SECRET_ACCESS_KEY'),
                      region_name=region_name)

    # Retrieve the CSV file from S3
    obj = s3.get_object(Bucket=bucket_name, Key=os.environ.get('S3_FILE_KEY'))
    data = obj['Body'].read()

    # Load CSV data into a Pandas DataFrame
    df = pd.read_csv(io.BytesIO(data))

    return df

"""
import boto3
import io

df = read_csv_from_s3(s3_bucket='loan_application_data-raw',
                      region_name=aws_region)
"""

# Data Types Inspection

In [ ]:
# Check features and data types and
df.info()

# Manual inspection

In [ ]:
df.head()

# Inspect values of categorical features

Check if company_type only contains AB as communicated in the document

In [ ]:
df['company_type'].unique()

# Domain-based Feature Selection

Remove `application_id` and `index` because they are just to track and find applications, it does not bring any information for the ML modeling

In [ ]:
df = df.drop(columns=['index', 'r_application_id'])

Remove `company_type` because it contains the same value for all entries

In [ ]:
df = df.drop(columns=['company_type'])

# Target Distribution Inspection

In [ ]:
sns.set(style="whitegrid")

ax = sns.countplot(x=df['default'], palette="Set3", hue=df['default'], legend=False)
sns.set(font_scale=1.5)
ax.set_xlabel('Loan Default')
ax.set_ylabel('Frequency')
fig = plt.gcf()
fig.set_size_inches(10, 5)

# Adding percentage labels on each bar
total = len(df['default'])
for p in ax.patches:
    percentage = '{:.1f}%'.format(100 * p.get_height() / total)
    x = p.get_x() + p.get_width() / 2
    y = p.get_height()
    ax.annotate(percentage, (x, y), ha='center', va='bottom')

plt.title('RAW Distribution of Loan Default')
plt.savefig(PATH_PLOTS_RAW+"raw_target_distribution.png")
plt.show()

Target distribution consists of mostly non-approved applications

# Values Formatting

#### Application and Incorporation Date

Models work better with numerical representations. Convert the incorporation date in the number of days from a reference date (1st January 1900)

In [ ]:
# Convert date from String to Datetime
df['applic_date'] = pd.to_datetime(df['applic_date'])
df['incorporation_date'] = pd.to_datetime(df['incorporation_date'])

# Fix Reference Date
reference_date = datetime.datetime(1900, 1, 1)

# Compute Number of Days from Reference Day
df['application_days'] = (df['applic_date'] - reference_date).dt.days.round().astype('Int64')
df['incorporation_days'] = (df['incorporation_date'] - reference_date).dt.days.round().astype('Int64')

# Delete Original Full Date Feature
df = df.drop(columns=['applic_date'])
df = df.drop(columns=['incorporation_date'])

#### Company Rating

`company_rating` could have negative values when value is not indicated. Trasform them into NA

In [ ]:
df.loc[df['company_rating'] < 0, 'company_rating'] = pd.NA

In [ ]:
# Check if any cleaning is needed also on the upper limit
max(df['company_rating'])

#### Person Scoring

`person_scoring`, like company rating, could have negative values when value is not indicated. Trasform them into NA

In [ ]:
df.loc[df['person_scoring'] < 0, 'person_scoring'] = pd.NA

In [ ]:
# Check if any cleaning is needed also on the upper limit
max(df['person_scoring'])

#### UC Risk Class

`uc_risk_class` could have "nr" values when value is not indicated. Trasform them into NA

In [ ]:
# Replace 'nr' (or any other string) with NaN and convert the column to int
df['uc_risk_class'] = pd.to_numeric(df['uc_risk_class'], errors='coerce').astype('Int64')

#### Look for other formatting

In [ ]:
df.describe()

# Inspect value distributions

In [ ]:
# Set the style to whitegrid
sns.set(style="whitegrid")

fig, axes = plt.subplots(nrows=5, ncols=2, figsize=(12, 12))

for i, column in enumerate(df.columns):
    row, col = divmod(i, 2)
    sns.histplot(df[column], ax=axes[i // 2, i % 2], bins=20, kde=True)
    axes[i // 2, i % 2].set_title(column)
    axes[i // 2, i % 2].set_xlabel('Value')
    axes[i // 2, i % 2].set_ylabel('Frequency')

plt.tight_layout()
plt.savefig(PATH_PLOTS_RAW+"raw_distributions.png")
plt.show()

From the plots I see that features such as `default`, `pre_contr_count`, `net_turnover`, and `max_late_1y` are highly skewed.
`person_scoring` and `company_rating` are more uniformly distributed.

From the plots I see that features do not have strange values to be formatted (e.g. -1 values to be converted to NA)

# Correlation-based Feature Selection

Correlated feature might not provide additional information and might even introduce multicollinearity. Compute a correlation matrix for the numerical features.

In [ ]:
# Do not include target
df_corr = df.drop(columns=['default'])

# Select only numerical features
features_num = df_corr.select_dtypes(include=['number']).columns

# Compute Correlation Matrix
correlation_matrix = df_corr[features_num].corr().abs()

# Select the upper triangle (correlation matrix is symmetric)
upper_triangle = correlation_matrix.where(np.triu(np.ones(correlation_matrix.shape), k=1).astype(bool))

# Select features with a correlation higher than 0.8
to_drop = [column for column in upper_triangle.columns if any(upper_triangle[column] > 0.8)]

print('Features with correlation higher than 0.8: ', to_drop)

In [ ]:
# Visualize Correlation Matrix with Heatmap
sns.set(style="whitegrid")
plt.figure(figsize=(12, 8))
sns.heatmap(upper_triangle, cmap="coolwarm", annot=True, fmt=".2f", vmin=0, vmax=1, cbar_kws={'label': 'Correlation'})
plt.title("Correlation Heatmap")
plt.tight_layout()

# Save the plot
plt.savefig(PATH_PLOTS_RAW+"correlation_heatmap.png")

plt.show()

No feature can be removed due to correlation.

# Final Summary

In [ ]:
df.info()

In [ ]:
df.describe()

# Save Data Locally

In [ ]:
# Check if the directory for processed data exists
if not os.path.exists(PATH_DATA_PREPROCESSED):
    # If it doesn't exist, create the directory
    os.makedirs(PATH_DATA_PREPROCESSED)

# Save processed data
df.to_csv(PATH_DATA_PREPROCESSED+'loan_application_data-preprocessed.csv', header=True, index=False)